# Idling Insights with the Data Science Package

Is idling a pervasive problem for your fleet? Do you have a specific question or hypothesis around idling in your fleet? Using the DSP can allow you to easily compare areas where your fleets compared to Geotab's known idling areas, or can help you add some context to your fleet's idling tendencies. 

In [0]:
import pandas as pd

We can first get all of the instances of idling in the past month. 
This query will give you the SerialNo of the vehicle that was idling, the duration, the time that idling started and ended, and the location (both latitude/longitude and Geohash)
In order to find the areas where vehicles in your fleet idle most often, you can use Geohash and compare them to Geotab's Idling Areas Dataset.

In [0]:
projectname='PUT PROJECT NAME HERE'

In [0]:
query_I= """
With FIdle As 
    (Select Geohash, Sum(Seconds_Idling) as Idling_InSec  
       From 
       (Select * 
          From `geotab-dsp-$name.Exceptions.Idling` 
         Where _PartitionTime between Timestamp(Date_sub(current_Date(), interval 31 Day)) 
               and Timestamp(Date_sub(current_Date(), interval 1 DAY)))
         Group by Geohash)

Select * 
  From `geotab-public-intelligence.UrbanInfrastructure.IdlingAreas` I 
 Inner Join FIdle F 
    On I.Geohash=Substr(F.Geohash,1,7)
"""

In [0]:
df_i = pd.read_gbq(query_I.replace('$name', projectname), project_id='geotab-dsp-'+projectname, dialect='standard')

We also might want to add some additional context around idling within your fleet. You can easily summarize idling tendencies by vehicle type or vocation, or look at the dates or times of day that vehicles tend to idle. The query below allows you to quickly see differences in idling between vehicle types, vocation, date, and serial number. 

In [0]:
query_IS= """
With FIdle As (
  Select SerialNo, StartDateTime, StopDateTime, Duration
    From `geotab-dsp-$name.Exceptions.Idling` 
    Where _PartitionTime between Timestamp(Date_sub(current_Date(), interval 31 Day)) 
          and Timestamp(Date_sub(current_Date(), interval 1 DAY)))

, Trips As ( 
   Select Serialno, TripId, StartTime, StopTime, DrivingDuration 
     From  `geotab-dsp-$name.Trips.Trips` 
    Inner Join FIdle On SerialNo
    Where  _PartitionTime between Timestamp(Date_sub(current_Date(), interval 31 Day)) 
          and Timestamp(Date_sub(current_Date(), interval 1 DAY))
          and StartDateTime between StartTime and StopTime)

, SwV As (
    Select T.*, V.Vin 
     From 
      (Select Vin, SerialNo 
        From `geotab-dsp-$name.Interpolated.Vin` 
       Where Datefrom< Timestamp(Date_sub(current_Date(), interval 31 Day))
      ) V 
   Inner Join Trips T 
     On F.SerialNo=V.SerialNo)

, Decode AS (
  Select Vin, Make, Model, Year, VehicleType, WeightClass, S.*, 
    CASE WHEN VehicleType LIKE 'Passenger' THEN 'Car',
         ELSE VehicleType LIKE 'MPV' THEN 'MPV',
         ELSE VehicleType LIKE 'Truck' AND (WeightClass LIKE '%Class 8%' 
           OR WeightClass LIKE '%Class 7%') THEN 'HDT',
         ELSE VehicleType LIKE 'Truck' AND (WeightClass LIKE '%Class 6%' 
           OR WeightClass LIKE '%Class 5%' OR WeightClass LIKE '%Class 4%') 
         THEN 'MDT',
         ELSE VehicleType LIKE 'Truck' AND (WeightClass LIKE '%Class H%' 
           OR WeightClass LIKE '%Class G%' OR WeightClass LIKE '%Class E%' 
           OR WeightClass LIKE '%Class C%' OR WeightClass LIKE '%Class D%' 
           OR WeightClass LIKE '%Class 3%' OR WeightClass LIKE '%Class F%' 
           OR WeightClass LIKE '%Class B%' OR WeightClass LIKE '%Class E%' 
           OR WeightClass LIKE '%Class 1%' OR WeightClass LIKE '%Class A%') 
         THEN 'LDT' ELSE 0 END AS Veh_Type
  From `geotab-dsp-$name.VinDecode` V 
  Inner Join Swv S Using(Vin))

Select Serialno, TripId, Duration as Minutes_Idling, StartTime, StopTime, DrivingDuration as DriveTime, Veh_Type
  From Decode
"""

In [0]:
df_IS = pd.read_gbq(query_I.replace('$name', projectname),project_id='geotab-dsp-'+projectname, dialect='standard')

A sample dashboard of this information is available here: https://datastudio.google.com/s/o57fDfsvdEA